In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
import yfinance as yf

In [2]:
tickers=['GOOG', '^GSPC']
yf.pdr_override()
data=pd.DataFrame()
for ticker in tickers:
    data[ticker]=wb.get_data_yahoo(ticker, start='2010-01-01')['Adj Close']

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:
data.head()

,GOOG,^GSPC
Date,,
2010-01-04,15.610239,1132.989990
2010-01-05,15.541497,1136.520020
2010-01-06,15.149715,1137.140015
2010-01-07,14.797037,1141.689941
2010-01-08,14.994298,1144.979980


In [4]:
returns=np.log(data/data.shift(1))
returns.head()

,GOOG,^GSPC
Date,,
2010-01-04,NaN,NaN
2010-01-05,-0.004413,0.003111
2010-01-06,-0.025532,0.000545
2010-01-07,-0.023555,0.003993
2010-01-08,0.013243,0.002878


In [5]:
cov=returns.cov()*252
cov.head()

,GOOG,^GSPC
GOOG,0.074348,0.033272
^GSPC,0.033272,0.030640


In [6]:
c_with_market=cov.iloc[0,1]
c_with_market

0.033271914610566056

In [7]:
market_var=returns['^GSPC'].var()*252
market_var

0.030640027827970404

In [8]:
goog_beta=c_with_market/market_var
goog_beta

1.085897010191129

In [9]:
riskfree=0.025
riskpremium=(returns['^GSPC'].mean()*252)-riskfree
goog_capm_return=riskfree+ goog_beta*(riskpremium)
goog_capm_return

0.1135857343325074

In [10]:
log_return=np.log(data/data.shift(1))
sharpe_goog=(goog_capm_return - riskfree)/((log_return['GOOG'].std()*252)**0.5)
sharpe_goog

0.042579035789864765

In [11]:
def import_data(tickers, start):
    yf.pdr_override()
    data=pd.DataFrame()
    for ticker in tickers:
        data[ticker]=wb.get_data_yahoo(ticker, start)['Adj Close']
    return data

In [12]:
def compute_beta(data, stock, market):
    ret=np.log(data/data.shift(1))
    cov=ret.cov()*252
    cov_w_m=cov.iloc[0,1]
    market_var=ret[market].var()*252
    return cov_w_m/market_var

In [13]:
def capm(data, riskfree, beta, market):
    ret=np.log(data/data.shift(1))
    riskprem=ret[market].mean()*252-riskfree
    capm=riskfree+beta*riskprem
    return capm

In [14]:
def sharpe(data, riskfree, capm, stock):
    ret=np.log(data/data.shift(1))
    sharpe_stock=(capm-riskfree)/((ret[stock].std()*252)**0.5)
    return sharpe_stock

In [15]:
def stock_capm(stock, market='^GSPC', start='2010-01-01', riskfree=0.025):
    data=import_data([stock, market], start)
    beta=compute_beta(data, stock, market)
    cal_capm=capm(data, riskfree, beta, market)
    cal_sharpe=sharpe(data, riskfree, cal_capm, stock)
    print(f"Beta {beta}\nCAPM Expected Return {cal_capm}\nSharpe {cal_sharpe}")
    return [beta, cal_capm, cal_sharpe]

In [16]:
res=stock_capm('GOOG', '^GSPC')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
Beta 1.085897010191129
CAPM Expected Return 0.1135857343325074
Sharpe 0.042579035789864765
